## First Step: create viewsheds based on 6 image locations
1. Load Data
    * DEMS
    * GPS csv file
2. Convert csv file to coordinates
3. Convert GPS coordinates (a feature layer) to shp
4. Create individual shps for each image point
    * Extract relevant values for names (names for new shapefiles)
    * Create 6 copies of the GOS shp (conatining all data) and delete all rows expt 1 point (then save shp)
5. Use Viewshed() to calculate viewshed rasters for each image


<i>Please Note this Notebook was completed within an ArcGIS Pro environment with access to Arcpy</i>

In [1]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *
from pathlib import Path

In [5]:
########################################## Creating Variables for Project ##################################################
# adjust as to accurate location of files
GPS_csv_filepath = r"C:\Users\runac\Downloads\Fall_2022\Programming\project_idea\JupNotebooks\GPS_metadata.csv"
DEM_filepath = r"D:\Fall_2022\Programming\Project\Data\30DEM\USGS_1_n49w114_20210607.tif"
output_filepath = "D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb"
Num_image_Coords = 6

In [6]:
def Creating_file_names(path, filename_str, length):
    """
    Provide the path where you'd like to save each file, and the naming string used to name resulting files
    Filenmae_str needs to be formatted as "\<filename>"
    Returns a sequential numbered list of filename paths for the provided length (length = number of image coordinates being used to create viewsheds)
    """
    path_ = path
    list_ = []
    for number in range(length): 
        filename = filename_str + str(number + 1)
        list_.append(path_ + filename)
    
    return list_

In [24]:
#################### Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile #########
##create a def to be called later that extract the coordinates for each image.shp
def image_coords(path_to_image):
    '''
    Given the shapefile path you can extract the points coorndinates. 
    Returns a arcpy Point
    '''
    with arcpy.da.SearchCursor(path_to_image, ['*']) as cursor:
        for row in cursor:
            #assumption, shp is 1 point, thus only returning 1 point value, if shp with many pts provided will return last pt
            cp = arcpy.Point(row[1][0], row[1][1])
    return cp

def creating_polylines(filepath, anchor_point, connection_points): 
    """
    creates lines from each coordinate (image<#>.shp) to the points plotted on the minimum bounding circle 
    created around the viewshed. 
    filepath: insert desired name for each polyline file created with entire filepath (to .gdb)
    anchor_point: filepath to shapefile for each respective image's coordinates
    connection_points: filepath to the points created on the minimum bounding circle
    Returns a new shapefile made of polylines
    """
    # ##creating new file name and shapefile for polylines
    new_fc = filepath
    name = filepath[(len(output_filepath)+1):len(filepath)]
    #matching the spatial refernce of the DEM file
    sr = arcpy.SpatialReference(4269) 
    arcpy.management.CreateFeatureclass(output_filepath, name, 'POLYLINE', spatial_reference = sr)
    #ADDing a column that designates degree (I believe arc designates N as 0 and rotates clockwise)
    arcpy.management.AddField(name, "Degree", "DOUBLE", None, None, None, "Degree", "NULLABLE", "NON_REQUIRED", '')

    percentage = 0
    
    ##Creating a line for each point on perimeter to corresponding image
    #looping through perimeter points
    with arcpy.da.SearchCursor(connection_points, ['*']) as cursor:
        for row in cursor:
            list_points = arcpy.Array([])
            #calulating the degree sequentially clockwise (adjust percentage if GeneratePointsAlongLines() uses differnt percentage)
            degree = (percentage/100)*360
            
            #declaring start (image location) & end (pts on circle) points for each line
            cp1 = image_coords(anchor_point) #pulling the same image point
            cp2 = arcpy.Point(row[1][0], row[1][1])
            list_points.append(cp2)
            list_points.append(cp1)
            
            #assignng spatial reference (should check to make sure its accurate)
            spatial_reference = arcpy.SpatialReference(4326)
            polyline = arcpy.Polyline(list_points, spatial_reference)
           
            #inserting new line to new feature class
            icur = arcpy.da.InsertCursor(new_fc, ["SHAPE@", "Degree"])
            icur.insertRow([polyline, degree])
            del icur
            
            #updating percentage for each line
            percentage += 0.2


In [16]:
########################################## Loading DEM and GPS data ###########################################################
#(steps 1-4)
##relevant rasters needed for the viewshed...
DEM1 = Raster(DEM_filepath)

#convert the csv table to points (run once)
arcpy.management.XYTableToPoint(GPS_csv_filepath, 
                                r"GPS_metadata_XYTableToPoint", 
                                "decimal_Long", 
                                "decimal_Lat",
                                "GPSAltitude", 
                                'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],VERTCS["WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision')
###converting GPS data to a shp
arcpy.conversion.FeatureClassToShapefile("GPS_metadata_XYTableToPoint", output_filepath)

##creating a list that stores the name of each image file (used later to create individual shp for each point)
List_shp = []
List_names = []
with arcpy.da.UpdateCursor("GPS_metadata_XYTableToPoint", ['Field1']) as cursor:
    for row in cursor:
        print(row[0])
        name = row[0].split('.')
        List_shp.append(f'{name[0]}.shp')
        List_names.append(row[0])
        
##copying featureclass and converting to individual shpfiles for each point
data = output_filepath + "\GPS_metadata_XYTableToPoint"
count = 0
for name in List_shp:
    #copying original file 
    arcpy.management.CopyFeatures(data, name)
    #creating shpfiles with 1 point by deleting all features excepts for 1 row
    with arcpy.da.UpdateCursor(name, ['Field1','decimal_La','decimal_Lo', 'GPSAltit_1']) as cursor:
        for row in cursor:
            if row[0] != List_names[count]:
                cursor.deleteRow()
    count += 1

image1.HEIC
image2.HEIC
image3.HEIC
image4.JPG
image5.JPG
image8.JPG


In [17]:
############### Creating a list of names to use for viewsheds, based on the number of images processed ###################
# (step 5)
###declaring lists for names and creating files for Viewshed Outputs
List_Raster_V_names = Creating_file_names(output_filepath, "\outvwdshd", Num_image_Coords)
List_outnames = Creating_file_names(output_filepath, "\outViewshed", Num_image_Coords)

###################################################### VIEWSHED ############################################################
for shp, out_file, vwshd in zip(List_shp, List_outnames, List_Raster_V_names):
    # Execute Viewshed (DON'T NEED 'List_Raster_V_names')
    outViewshed = Viewshed(DEM1, shp, 1, "FLAT_EARTH", 0.13)
    # Save the output 
    outViewshed.save(out_file)
    name = vwshd[(len(output_filepath)+1):len(vwshd)]
    print("saving", name)


saving outvwdshd1
saving outvwdshd2
saving outvwdshd3
saving outvwdshd4
saving outvwdshd5
saving outvwdshd6


## Next big step: extracting elevation values (maxes) in a 360 view around image points. 

1. Viewshed raster to Polygon
2. Create a minimum bounding circle on vectorized viewshed raster
3. Generate points on perimeter of circle (min bounding region)
4. Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile.
5. Use vectorized viewshed raster to mask DEM (resulting in a viewshed shaped DEM with real elevation values)
6. Extract the cell values from the clipped viewshed DEM with the Polylines. (HERE CURRENTLY!!)
    * For each line store the largest DEM value and save values in a table
7. Plot table

In [18]:
################################################## Viewshed raster to Polygon ########################################
##vectorizing resulting viewshed rasters

#list of names of the viewshed tifs
vwshd_tifs = Creating_file_names(output_filepath, "\outViewshed", Num_image_Coords)
#list of names to be used for vectorized viewsheds
ras_to_polys = Creating_file_names(output_filepath, "\Poly_outView", Num_image_Coords)

###### Looping over "ras_to_polys" to convert ALL viewsheds to polygons
for tif, poly in zip(vwshd_tifs, ras_to_polys):
    #isolating names of tif file from rest of path
    name = tif[(len(output_filepath)+ 1):len(tif)]
    #converting rasters to polygons
    arcpy.conversion.RasterToPolygon(name, 
                                     poly, 
                                     "SIMPLIFY", 
                                     "Value", 
                                     "SINGLE_OUTER_PART", 
                                     None)

# #deleting polygons with gridcode = 0 (represents areas viewer cannot see)
for v in ras_to_polys: 
    with arcpy.da.UpdateCursor(v, ['*']) as cursor:
        for row in cursor:
            if row[3] == 0:
                cursor.deleteRow()
    print("Saving & Updating:", v[(len(output_filepath)+1):len(v)])

Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView1
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView2
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView3
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView4
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView5
Saving & Updating: D:\Fall_2022\Programming\Project\Viewshed_MT_clean\Viewshed_MT_clean.gdb\Poly_outView6


In [19]:
########################### create a min bounding circle on vectorized viewshed raster #######################
# creating list of names for minimum bounding circles
min_bound_names = Creating_file_names(output_filepath, "\B_outView", Num_image_Coords)

# creating minimum bounding circle around viewshds
for poly_vwshd, min_bound in zip(ras_to_polys, min_bound_names):
    arcpy.management.MinimumBoundingGeometry(poly_vwshd, 
                                         min_bound, 
                                         "CIRCLE", 
                                         "ALL")
    print("Saving", min_bound[(len(output_filepath)+1):len(min_bound)])

Saving \B_outView1
Saving \B_outView2
Saving \B_outView3
Saving \B_outView4
Saving \B_outView5
Saving \B_outView6


In [20]:
############################## Generate points on perimeter of circle (min bounding region) ######################
# creating list of names for points on minimum bouding circles
pts_names = Creating_file_names(output_filepath, "\Pts_View", Num_image_Coords)

# creating pts on minimum bounding circle
for circle, pts in zip(min_bound_names, pts_names):
    arcpy.management.GeneratePointsAlongLines(circle, 
                                          pts, 
                                          "PERCENTAGE",  
                                          Percentage=0.2, 
                                          Include_End_Points='END_POINTS')
    
    print("Saving", pts[(len(output_filepath)+1):len(pts)])

    ##takes apporx 2-3 mins to execute

Saving Pts_View1
Saving Pts_View2
Saving Pts_View3
Saving Pts_View4
Saving Pts_View5
Saving Pts_View6


In [25]:
#################### Connect image.shp point to each of the points on circle perimeter creating a new polyline shapefile ##########

# creating paths and names for new polyline files 
polyline_names = Creating_file_names(output_filepath, "\View_lines", Num_image_Coords)

# polyline_names : recently declared above
# pts_names : calling this already made list of minimum bounding circle pts
# List_shp : calling this already made list of image locations 

#Running the function that'll create polylines for each image and corresponding viewshed points
for polyline, image, pts, in zip(polyline_names, List_shp, pts_names):
    print("Creating:", polyline[(len(output_filepath) + 1):len(polyline)])
    creating_polylines(polyline, image, pts)
    
#takes 1.5 mins to make lines when p = 0.2

Creating: View_lines1
Creating: View_lines2
Creating: View_lines3
Creating: View_lines4
Creating: View_lines5
Creating: View_lines6


In [28]:
########################################## Use vectorized viewshed raster to mask DEM ###################################
# creating paths and names for new clipped DEM of viewsheds 
c_DEM_names = Creating_file_names(output_filepath, "\Clipped_DEM_Vwshd", Num_image_Coords)

#declaring the spatial extent of the DEM1 (used to help clip the DEM to the viewshed output)
extent = DEM1.extent
xmin = extent.XMin
ymin = extent.YMin
xmax = extent.XMax
ymax = extent.YMax
spatial_extent = str(xmin) + " " + str(ymin) +  " " + str(xmax) +  " " +str(ymax)

#creating a clipped extent of DEM for each image
for vwshd, clip_dem in zip(ras_to_polys, c_DEM_names):
    print("Clipping:", clip_dem[(len(output_filepath)+1):len(clip_dem)])
    poly_vwshd = vwshd[(len(output_filepath)+1):len(vwshd)]
    arcpy.management.Clip(DEM1, 
                          spatial_extent,
                          clip_dem, 
                          poly_vwshd, 
                          "-999999", 
                          "ClippingGeometry", 
                          "NO_MAINTAIN_EXTENT")

Clipping: Clipped_DEM_Vwshd1
Clipping: Clipped_DEM_Vwshd2
Clipping: Clipped_DEM_Vwshd3
Clipping: Clipped_DEM_Vwshd4
Clipping: Clipped_DEM_Vwshd5
Clipping: Clipped_DEM_Vwshd6


In [32]:
########################################## Extract the Elevation values from the clipped viewshed DEM with the Polylines ############
# creating paths and names for new INTIGER-(datatype) clipped DEM of viewsheds 
int_DEM_names = Creating_file_names(output_filepath, "\Int_clip_DEM_Vwshd", Num_image_Coords)
poly_pixels_vw = Creating_file_names(output_filepath, "\Poly_Int_Clip_DEM", Num_image_Coords)
max_elevation = Creating_file_names(output_filepath, "\lines_max_elev", Num_image_Coords)

##convert elevation datatype from decimal to integer (looses some data)
for clip_dem, int_dem in zip(c_DEM_names, int_DEM_names):
    print("Saving", int_dem[(len(output_filepath)+1):len(int_dem)])
    out_raster = arcpy.ia.Int(clip_dem[(len(output_filepath)+1):len(clip_dem)]); out_raster.save(int_dem)

##convert raster to polygons (s.t. each raster pixel is a polygon)
for int_dem, poly_dem in zip(int_DEM_names, poly_pixels_vw):
    arcpy.conversion.RasterToPolygon(int_dem[(len(output_filepath)+1):len(int_dem)], 
                                     poly_dem, 
                                     "NO_SIMPLIFY", 
                                     "Value", 
                                     "SINGLE_OUTER_PART", 
                                     None)

##spatial join between pixels (as polygons) and polylines : whilst saving only max elevations for each polyline
##this join returns the max 'gridcode' ('elevation value') that intersects with a line!!
##need to figure out how to apply 'field mapping' for each vectorized-pixel viewshed..?
for lines, poly_dem, elev in zip(polyline_names, poly_pixels_vw, max_elevation):
    arcpy.analysis.SpatialJoin(lines[(len(output_filepath)+1):len(lines)], 
                               poly_dem[(len(output_filepath)+1):len(poly_dem)], 
                               elev, 
                               "JOIN_ONE_TO_ONE", 
                               "KEEP_ALL", 
                               'Degree "Degree" true true false 19 Double 0 0,First,#,view1_lines_b,Degree,-1,-1;gridcode "gridcode" true true false 4 Long 0 0,Max,#,Poly_Int_Clip_DEM1,gridcode,-1,-1', 
                               "INTERSECT", 
                               None, 
                               '')
    print("Saving Max Elevation value")

Saving Int_clip_DEM_Vwshd1
Saving Int_clip_DEM_Vwshd2
Saving Int_clip_DEM_Vwshd3
Saving Int_clip_DEM_Vwshd4
Saving Int_clip_DEM_Vwshd5
Saving Int_clip_DEM_Vwshd6
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value
Saving Max Elevation value


In [42]:
##exported the shp as a csv to view in LATIS
out_path = r"D:\Fall_2022\Programming\Project\Viewshed_MT_clean"
csv_names = Creating_file_names(out_path, "\lines_max_elev", Num_image_Coords)

for elev, csv in zip(max_elevation, csv_names):
    name = csv + ".csv"
    print("Saving:", name[(len(out_path)+1):(len(csv)+4)])
    arcpy.conversion.ExportTable(elev[(len(output_filepath)+1):len(elev)], 
                                 name, 
                                 '', 
                                 "NOT_USE_ALIAS", 
                                 'Join_Count "Join_Count" true true false 4 Long 0 0,First,#,view1_lines_b_SpatialJoin2,Join_Count,-1,-1;TARGET_FID "TARGET_FID" true true false 4 Long 0 0,First,#,view1_lines_b_SpatialJoin2,TARGET_FID,-1,-1;Degree "Degree" true true false 8 Double 0 0,First,#,view1_lines_b_SpatialJoin2,Degree,-1,-1;gridcode "gridcode" true true false 4 Long 0 0,First,#,view1_lines_b_SpatialJoin2,gridcode,-1,-1', 
                                 None)

Saving: lines_max_elev1.cs
Saving: lines_max_elev2.cs
Saving: lines_max_elev3.cs
Saving: lines_max_elev4.cs
Saving: lines_max_elev5.cs
Saving: lines_max_elev6.cs


##### After successfully extrating the Max elevation values for each polyline, I need to measure distance from observer/image pt to location of Max elevation pixel. I then need to create new values where max elevation is divided by distance (to accurately depict a mountain ridgeline w.r.t. distance) 



In [ ]:
################################### Extract Distances from image pt to each max elevation for each line ###############################
## An additional Spatal Join where pixels (as polygons) Ar kept only if they intersect each polyline AND have the MAX elevation...(still thinking this through)

### Scratch Work